In [ ]:
import json
import pandas as pd
import geopandas as gpd
import altair as alt

import vega_datasets
from ast import literal_eval

import numpy as np
import folium
from folium import Map, FeatureGroup, Marker, LayerControl
from folium.plugins import MeasureControl
import branca.colormap as cm

from localtileserver import get_folium_tile_layer
from localtileserver import TileClient

In [ ]:
def array_eval(x):
    array_str = ','.join(x.split())
    array_str = array_str.replace('[,', '[')
    array_str = array_str.replace('\n', '')
    return np.array(literal_eval(array_str))

gdf = gpd.read_file(r"C:\BERATools\Test-data\footprint-fixed-avg.geojson")
#gdf = gpd.read_file(r"C:\BERATools\Test-data\original\footprint_original_centerline.geojson")   
gdf['sampling_widths'] = gdf['sampling_widths'].apply(array_eval)

In [ ]:
def add_histgram(widths):
    width_max, width_min = max(widths), min(widths)
    widths_pd = pd.DataFrame({'sampling_widths': widths})
    width_max, width_min

    hist = alt.Chart(widths_pd).mark_bar().encode(
    alt.X("sampling_widths", bin=alt.Bin(extent=[width_min, width_max], step=1.0)),
    y='count()',
    )

    vega_lite = folium.VegaLite(
        hist,
        width="100%",
        height="100%",
    )

    return vega_lite


Prepare raster layers

In [ ]:
# First, create a tile server from local raster file
tile_client = TileClient(r"C:\BERATools\Test-data\CHM.tif")
t = get_folium_tile_layer(tile_client)
t.control=True

# Study sites
# tile_4203 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4203_transparent_mosaic_Shifted.tif")
# t_4203 = get_folium_tile_layer(tile_4203)
# t_4203.control=True

# tile_4207 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4207_transparent_mosaic_Shifted.tif")
# t_4207 = get_folium_tile_layer(tile_4207)
# t_4207.control=True

# tile_4209 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4209_transparent_mosaic_Shifted.tif")
# t_4209 = get_folium_tile_layer(tile_4209)
# t_4209.control=True

In [ ]:
# Create two maps.
m = folium.Map(
    location=[56.201831,-110.672306],
    zoom_start=14,
    position="absolute",
    left="0%"
)

colormap = cm.linear.Set1_09.scale(min(gdf.avg_width), max(gdf.avg_width)).to_step(10)
colormap.caption = "A colormap caption"
m.add_child(colormap)

featuregrp_fixed_fp = FeatureGroup(name='Fixed widthe footprint')

for index, row in gdf.iterrows(): 
    widths = row['sampling_widths']
    polygon = folium.GeoJson(
        row['geometry'],
        style_function=lambda feature: {
            "fillColor": colormap(row['avg_width']),
            "color": "blue",
            "weight": 1,},
    )
          
    popup = folium.Popup()
    vega_lite = add_histgram(widths)
    vega_lite.add_to(popup)
    popup.add_to(polygon)

    tooltip = folium.Tooltip(row['avg_width'])
    tooltip.add_to(polygon)

    featuregrp_fixed_fp.add_child(polygon)
    #polygon.add_to(m)

In [ ]:
m.add_child(t)
# m.add_child(t_4203)
# m.add_child(t_4207)
# t_4209.add_to(m)

featuregrp_fixed_fp.add_to(m)
m.add_child(MeasureControl())
folium.LayerControl().add_to(m)
m

In [ ]:
gdf.avg_width